<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1a_SLSTR_data_access_Data_Store.ipynb"><< Accessing SLSTR data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_SLSTR_coverage.ipynb">Determining SLSTR product coverage >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SLSTR: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook, and will retrieve the data required here.
  - **<a href="./1_1a_SLSTR_data_access_Data_Store.ipynb" target="_blank">1_1a_SLSTR_data_access_Data_Store.ipynb</a>** if using the Data Store for data access
    <br><br>**OR**<br><br>
  - **<a href="./1_1b_SLSTR_data_access_HDA.ipynb" target="_blank">1_1b_SLSTR_data_access_HDA.ipynb</a>** if using WEkEO for data access
    
</div>
<hr>

# 1.2 Understanding SLSTR product structure

### Data used

| Product Description | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| Sentinel-3 SLSTR level-1B | EO:EUM:DAT:0411 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SL_1_RBT___NTC?query=SLSTR&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SL_1_RBT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASL_1_RBT___&initial=1" target="_blank">link</a> |
| Sentinel-3 SLSTR level-2 (NTC) | EO:EUM:DAT:0412 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SL_2_WST___NTC?query=0412&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SL_2_WST___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASL_2_WST___&initial=1" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* What the SAFE format is
* What components are inside a SAFE format file from SLSTR
* What variables are in present in each component for SLSTR data files (level-1B and level-2)

### Outline

Data from all the Sentinel satellites operated under the European Commissions Copernicus Programme are delievered in "**SAFE format**". The Sentinel-SAFE format is a specific variation of the Standard Archive Format for Europe (SAFE) format specification designed for the Sentinel satellite products.  It is based on the XML Formatted Data Units (XFDU) standard under development by the Consultative Committee for Space Data Systems (CCSDS). Sentinel-SAFE is a profile of XFDU, and it restricts the XFDU specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between ground segment facilities.


Each product package includes:

* a manifest file containing a metadata section and a data object section (an xml file).

* measurement data files (NetCDF-4 format)

* annotation data files, if defined (NetCDF-4 format)

The product package can exist as a directory in a filesystem, zipped folder or tarball.

The naming of the Sentinel-SAFE files follows a specific convention that you can learn about [here](https://eumetsatspace.atlassian.net/wiki/spaces/DPF/pages/edit-v2/1597702263?). 

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Querying SLSTR file structure (Level-1B)](#section1)
 2. [Querying SLSTR file structure (Level-2)](#section2)
 3. [Applying your knowledge](#section3)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import os                                     # a library that allows us access to basic operating system commands
import glob                                   # a library that aids in searching for files
from IPython.display import display, Markdown # a library that helps us display HTML and markdown
import xarray as xr                           # a library that supports the use of multi-dimensional arrays in Python
import xml.etree.ElementTree as ET            # a library that helps us parse XML files
import warnings                               # a library that helps us manage warnings
warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Querying SLSTR file structure (Level-1B)
[Back to top](#TOC_TOP)

</div>

First we will create a variable which holds the path for the file we are interested in, in this case the level-1B OLCI file that we downloaded in either <a href="./1_1a_SLSTR_data_access_Data_Store.ipynb" target="_blank">1_1a_SLSTR_data_access_Data_Store.ipynb</a> or <a href="./1_1b_SLSTR_data_access_HDA.ipynb" target="_blank">1_1b_SLSTR_data_access_HDA.ipynb</a>.

In [2]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_SL_1_RBT____20220209T220650_20220209T220950_20220211T045404_0179_082_001_0540_MAR_O_NT_004.SEN3')

Next, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder. For Sentinel-3, this manifest is always called **xfdumanifest.xml** and contains very useful information about the nature of the contents of the SAFE format product.

In [3]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file in to a Python object format called a dictionary. This will make it easier for us to interact with the product in the next step.

In [4]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** ./viscal.nc

*File info: VISCAL data obtained from input VISCAL ADF*

**File name:** ./cartesian_an.nc

*File info: Full resolution cartesian coordinates for the A stripe grid, nadir view*

**File name:** ./cartesian_ao.nc

...

*File info: Time annotations for the A stripe grid*

**File name:** ./time_bn.nc

*File info: Time annotations for the B stripe grid*

**File name:** ./time_in.nc

*File info: Time annotations for the 1 KM grid*

The next section of code will display each of the components within the product. Scrolling through you can see the main groups of variables, including some you may already know or be interested in working with.

In [5]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** F1_BT_fn.nc

<xarray.Dataset>
Dimensions:                 (rows: 1202, columns: 1500, orphan_pixels: 187)
Dimensions without coordinates: rows, columns, orphan_pixels
Data variables:
    F1_BT_fn                (rows, columns) float32 ...
    F1_BT_orphan_fn         (rows, orphan_pixels) float32 ...
    F1_exception_fn         (rows, columns) uint8 ...
    F1_exception_orphan_fn  (rows, orphan_pixels) uint8 ...
Attributes: (12/17)
    absolute_orbit_number:  31158
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          20220211T045404Z
    history:                  2022-02-11T04:54:04Z: PUGCoreProcessor /data/ip...
    institution:            MAR
    ...                     ...
    source:                 IPF-SL-1 06.19
    start_offset:           44066
    start_time:             2022-02-09T22:06:49.623975Z
    stop_time:              2022-02-09T22:09:49.616377Z
    title:                  S3 SLSTR L1 Radiance and Brightness Temperatures ...
    track_offset:           998

**Filename:** F1_BT_fo.nc

<xarray.Dataset>
Dimensions:                 (rows: 1202, columns: 900, orphan_pixels: 112)
Dimensions without coordinates: rows, columns, orphan_pixels
Data variables:
    F1_BT_fo                (rows, columns) float32 ...
    F1_BT_orphan_fo         (rows, orphan_pixels) float32 ...
    F1_exception_fo         (rows, columns) uint8 ...
    F1_exception_orphan_fo  (rows, orphan_pixels) uint8 ...
Attributes: (12/17)
    absolute_orbit_number:  31158
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          20220211T045404Z
    history:                  2022-02-11T04:54:04Z: PUGCoreProcessor /data/ip...
    institution:            MAR
    ...                     ...
    source:                 IPF-SL-1 06.19
    start_offset:           44066
    start_time:             2022-02-09T22:06:49.623975Z
    stop_time:              2022-02-09T22:09:49.616377Z
    title:                  S3 SLSTR L1 Radiance and Brightness Temperatures ...
    track_offset:           450

**Filename:** F1_quality_fn.nc

...

<xarray.Dataset>
Dimensions:                (one: 1, rows: 2404)
Dimensions without coordinates: one, rows
Data variables: (12/13)
    NADIR_FIRST_PIXEL_b    (one) float64 ...
    Nadir_First_scan_b     (rows) float32 ...
    Nadir_Last_scan_b      (rows) float32 ...
    Nadir_Maximal_ts_b     (rows) datetime64[ns] ...
    Nadir_Minimal_ts_b     (rows) datetime64[ns] ...
    OBLIQUE_FIRST_PIXEL_b  (one) float64 ...
    ...                     ...
    Oblique_Last_scan_b    (rows) float32 ...
    Oblique_Maximal_ts_b   (rows) datetime64[ns] ...
    Oblique_Minimal_ts_b   (rows) datetime64[ns] ...
    PIXSYNC_b              (one) float32 ...
    SCANSYNC               (one) float64 ...
    time_stamp_b           (rows) datetime64[ns] ...
Attributes: (12/17)
    absolute_orbit_number:  31158
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          20220211T045404Z
    history:                  2022-02-11T04:54:04Z: PUGCoreProcessor /data/ip...
    institution:            MAR
    ...                     ...
    source:                 IPF-SL-1 06.19
    start_offset:           88131
    start_time:             2022-02-09T22:06:49.623975Z
    stop_time:              2022-02-09T22:09:49.616377Z
    title:                  S3 SLSTR L1 Radiance and Brightness Temperatures ...
    track_offset:           1996

**Filename:** time_in.nc

<xarray.Dataset>
Dimensions:                (one: 1, rows: 1202)
Dimensions without coordinates: one, rows
Data variables: (12/13)
    NADIR_FIRST_PIXEL_i    (one) float64 ...
    Nadir_First_scan_i     (rows) float32 ...
    Nadir_Last_scan_i      (rows) float32 ...
    Nadir_Maximal_ts_i     (rows) datetime64[ns] ...
    Nadir_Minimal_ts_i     (rows) datetime64[ns] ...
    OBLIQUE_FIRST_PIXEL_i  (one) float64 ...
    ...                     ...
    Oblique_Last_scan_i    (rows) float32 ...
    Oblique_Maximal_ts_i   (rows) datetime64[ns] ...
    Oblique_Minimal_ts_i   (rows) datetime64[ns] ...
    PIXSYNC_i              (one) float32 ...
    SCANSYNC               (one) float64 ...
    time_stamp_i           (rows) datetime64[ns] ...
Attributes: (12/17)
    absolute_orbit_number:  31158
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          20220211T045404Z
    history:                  2022-02-11T04:54:04Z: PUGCoreProcessor /data/ip...
    institution:            MAR
    ...                     ...
    source:                 IPF-SL-1 06.19
    start_offset:           44066
    start_time:             2022-02-09T22:06:49.623975Z
    stop_time:              2022-02-09T22:09:49.616377Z
    title:                  S3 SLSTR L1 Radiance and Brightness Temperatures ...
    track_offset:           998

**Filename:** viscal.nc

<xarray.Dataset>
Dimensions:               (one: 1, visible_detectors: 4, views: 2,
                           integrators: 2, swir_detectors: 8)
Dimensions without coordinates: one, visible_detectors, views, integrators,
                                swir_detectors
Data variables: (12/40)
    ANX_time              (one) object ...
    S1_BB1_SD             (visible_detectors, views, integrators) float32 ...
    S1_BB1_means          (visible_detectors, views, integrators) float32 ...
    S1_VISCAL_SD          (visible_detectors, views, integrators) float32 ...
    S1_VISCAL_means       (visible_detectors, views, integrators) float32 ...
    S1_slopes             (visible_detectors, views, integrators) float32 ...
    ...                    ...
    S6_VISCAL_means       (swir_detectors, views, integrators) float32 ...
    S6_slopes             (swir_detectors, views, integrators) float32 ...
    S6_solar_irradiances  (swir_detectors, views) float32 ...
    calibration_time      (one) object ...
    monitor_SD            (one) float32 ...
    monitor_mean          (one) float32 ...
Attributes:
    contact:              ops@eumetsat.int
    creation_time:        20220211T045404Z
    history:                2022-02-11T04:54:04Z: PUGCoreProcessor /data/ipf-...
    institution:          MAR
    netCDF_version:       4.2 of Jul  5 2012 17:07:43 $
    processing_baseline:  SL__L1_.004.04.00
    product_name:         S3A_SL_1_RBT____20220209T220650_20220209T220950_202...
    references:           
    source:               IPF-SL-1 06.19
    title:                S3 SLSTR L1 Radiance and Brightness Temperatures Pr...

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Querying SLSTR file structure (Level-2)
[Back to top](#TOC_TOP)

</div>

Now let's look at the level-2 SLSTR product that we downloaded previously.

In [6]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_SL_2_WST____20220209T213232_20220209T231332_20220211T074946_6059_081_385______MAR_O_NT_003.SEN3')

Again, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder.  

In [7]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file for this level-2 data product in to another dictionary. 

In [8]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** ./20220209213232-MAR-L2P_GHRSST-SSTskin-SLSTRA-20220211074946-v02.0-fv01.0.nc

*File info: L2P Data Set*

Finally, let's display the components of the product...

In [9]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** 20220209213232-MAR-L2P_GHRSST-SSTskin-SLSTRA-20220211074946-v02.0-fv01.0.nc

<xarray.Dataset>
Dimensions:                            (time: 1, nj: 40395, ni: 1500, channel: 3)
Coordinates:
    lat                                (nj, ni) float32 ...
    lon                                (nj, ni) float32 ...
  * time                               (time) datetime64[ns] 2022-02-09T21:32:32
Dimensions without coordinates: nj, ni, channel
Data variables: (12/22)
    adi_dtime_from_sst                 (time, nj, ni) float32 ...
    aerosol_dynamic_indicator          (time, nj, ni) float32 ...
    brightness_temperature             (channel, time, nj, ni) float32 ...
    dt_analysis                        (time, nj, ni) float32 ...
    dual_nadir_sst_difference          (time, nj, ni) float32 ...
    l2p_flags                          (time, nj, ni) int16 ...
    ...                                 ...
    sses_standard_deviation            (time, nj, ni) float32 ...
    sst_algorithm_type                 (time, nj, ni) int8 ...
    sst_dtime                          (time, nj, ni) timedelta64[ns] ...
    sst_theoretical_uncertainty        (time, nj, ni) float32 ...
    wind_speed                         (time, nj, ni) float32 ...
    wind_speed_dtime_from_sst          (time, nj, ni) float32 ...
Attributes: (12/48)
    Conventions:                CF-1.6, Unidata Observation Dataset v1.0
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    acknowledgment:             European Commission Copernicus Programme
    cdm_data_type:              swath
    comment:                    GHRSST SST L2P
    creator_email:              ops@eumetsat.int
    ...                         ...
    summary:                    Sentinel-3A SLSTR skin sea surface temperature
    time_coverage_end:          20220209T231331Z
    time_coverage_start:        20220209T213232Z
    title:                      Sentinel-3A SLSTR L2P SST dataset
    uuid:                       TBC
    westernmost_longitude:      -179.29269409179688

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Applying your knowledge
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-warning">

### Challenge:

What are differences you see between the level-1B and level-2 files? <div>

### Enter your solution here
* ...
* ...

* The products are substantially different in struture from level-1B to level-2.
* At level-1B, SLSTR RBT products contain various for each radiance channel and brightness temperature channel. At level-2, all variables are accumulated in a single file.
* At level-1 we have multiple grids defined, e.g. cartesian_??.nc
* At level-2, we have geophysical products, e.g. sea surface temperature.

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1a_SLSTR_data_access_Data_Store.ipynb"><< Accessing SLSTR data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_SLSTR_coverage.ipynb">Determining SLSTR product coverage >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean" target="_blank">View on GitLab</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>